# Mavlink Camera Client
> Mavlink experiments

[https://mavlink.io/en/mavgen_python/](https://mavlink.io/en/mavgen_python/)
[https://www.ardusub.com/developers/pymavlink.html](https://www.ardusub.com/developers/pymavlink.html)

https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_TYPE



In [ ]:
#| default_exp mavlink.cam_client

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
from UAV.imports import *   # TODO why is this relative import on nbdev_export?
import cv2

import numpy as np
from imutils import resize

import threading
import time
from pymavlink import mavutil
import threading
import logging
import UAV.params as params

# gi.require_version('Gst', '1.0')

In [ ]:
#| export
logging.basicConfig(format='%(asctime)-8s,%(msecs)-3d %(levelname)5s [%(filename)10s:%(lineno)3d] %(message)s',
                    datefmt='%H:%M:%S',
                    level=params.LOGGING_LEVEL)  # Todo add this to params
logger = logging.getLogger(params.LOGGING_NAME)

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
class CameraClient:
    """
    Mavlink Camera Client 
    """
    def __init__(self, connection_string, baudrate=57600):
        # Create the connection
        self.master = mavutil.mavlink_connection(connection_string, baud=baudrate)
        # we send a heartnbeat at the start as it's necessary to send data to server (via udpout) before server will send any data.
        self.send_heartbeat()
        print("Client heartbeat_sent")

        self.t = threading.Thread(target=self.listen, daemon=True)
        self.t.start()
        # Wait for the first heartbeat to get target system and component IDs
        print("Waiting for heartbeat")
        first_msg = self.master.recv_match(type='HEARTBEAT', blocking=True)
        print("Heartbeat from system (system %u component %u)" % (self.master.target_system, self.master.target_component))
        self.master.wait_heartbeat()
        print("Connected to MAVLink device")

    def trigger_camera(self, camera_id=1):
        # Use MAV_CMD_DO_DIGICAM_CONTROL to trigger the camera
        # This assumes the camera understands this MAVLink message
        self.master.mav.command_long_send(
            self.master.target_system,  # target_system
            self.master.target_component,  # target_component
            mavutil.mavlink.MAV_CMD_DO_DIGICAM_CONTROL,  # command
            0,  # confirmation
            camera_id,  # param1 (session)  or cam # (0 for all cams)
            1,  # param2 (trigger capture)
            0,  # param3 (zoom pos)
            0,  # param4 (zoom step)
            0,  # param5 (focus lock)
            0,  # param6 (shot ID)
            0,  # param7 (command ID)
        )
        print("sent Camera trigger")

    def start_streaming(self, camera_id=1):
        # Use MAV_CMD_VIDEO_START_STREAMING to start streaming video
        # This assumes the camera understands this MAVLink message
        self.master.mav.command_long_send(
            self.master.target_system,  # target_system
            self.master.target_component,  # target_component
            mavutil.mavlink.MAV_CMD_VIDEO_START_STREAMING,  # command
            0,  # confirmation
            camera_id,  # param1 (camera ID) Video Stream ID (0 for all streams, 1 for first, 2 for second, etc.)
            0,  # param2 (frame rate in Hz)
            0,  # param3 (0=disabled, 1=enabled)
            0,  # param4
            0,  # param5
            0,  # param6
            0,  # param7
        )
        print("sent Start streaming")

    def stop_streaming(self, camera_id=1):
        # Use MAV_CMD_VIDEO_STOP_STREAMING to stop streaming video
        # This assumes the camera understands this MAVLink message
        self.master.mav.command_long_send(
            self.master.target_system,  # target_system
            self.master.target_component,  # target_component
            mavutil.mavlink.MAV_CMD_VIDEO_STOP_STREAMING,  # command
            0,  # confirmation
            camera_id,  # param1 (camera ID) Video Stream ID (0 for all streams, 1 for first, 2 for second, etc.)
            0,  # param2
            0,  # param3
            0,  # param4
            0,  # param5
            0,  # param6
            0,  # param7
        )
        print("sent Stop streaming")

    def video_stream_info(self, camera_id=1):
        # Use MAV_CMD_REQUEST_MESSAGE to request video stream info
        # This assumes the camera understands this MAVLink message
        self.master.mav.command_long_send(
            self.master.target_system,  # target_system
            self.master.target_component,  # target_component
            mavutil.mavlink.MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION,  # command
            0,  # confirmation
            camera_id,  # param1 (camera ID) Video Stream ID (0 for all streams, 1 for first, 2 for second, etc.)
            0,  # param2 Video Stream ID (1 for first, 2 for second, etc.)
            0,  # param3
            0,  # param4
            0,  # param5
            0,  # param6
            0,  # param7
        )
        print("sent Request stream info")

    def send_heartbeat(self):
        self.master.mav.heartbeat_send(
            mavutil.mavlink.MAV_TYPE_GCS,  # type
            mavutil.mavlink.MAV_AUTOPILOT_INVALID,  # autopilot
            0,  # base_mode
            0,  # custom_mode
            mavutil.mavlink.MAV_STATE_ACTIVE,  # system_status
            3  # MAVLink version
        )

    def listen(self):
        print("Listening for MAVLink ...")


        while True:
            # Wait for a MAVLink message
            msg = self.master.recv_msg()
            if msg:
                print(msg)
            else:
                # print("No message")
                time.sleep(0.1)

    def close(self):
        self.master.port.close()
        print("Connection closed")

Mavlink Camera Client

In [ ]:
test_eq(1,1)

In [ ]:
client = CameraClient("udpout:localhost:14550")
# time.sleep(0.1)
# client.trigger_camera()
while True:
    client.video_stream_info(1)
    client.trigger_camera(1)
    client.start_streaming(2)
    time.sleep(5)

client.close()

Client heartbeat_sent
Listening for MAVLink ...
Waiting for heartbeat


KeyboardInterrupt: 

In [ ]:
assert False, "Stop here"

https://github.com/mavlink/MAVSDK/issues/1803

So I managed to change OpenHD in this regard.
No idea why I had such a hard time wrapping my head around, but now it works the following:
OpenHD binds port 127.0.0.1:14551 and listens on 127.0.0.1:14550
AND
instead of using sendto() with a unbound port (which then in turn means the sender port can be anything) messages are sent with sendto() from the bound port (the same that is used for listening).

So messages from OpenHD to mavsdk go the following:
OpenHD (out) via 127:0:0:1:14551 sent to 127:0:0:0:1:14550

So when mavsdk receives the first message, the sender address::port is 127:0:0:1:14551 and mavsdk can send the messages back to 127:0:0:1:14551.

https://julianoes.com/
The ports are not symmetrical! QGC listens on local port 14550 and sends UDP packets back to wherever messages came from.



In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()